<a href="https://colab.research.google.com/github/payalmohapatra/custome-kws-edge-impulse/blob/main/CustomDataset_KWS_mHealth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

### Import packages
Clone the TensorFlow Github Repository, which contains the relevant code required to run this code.


In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
!wget https://github.com/tensorflow/tensorflow/archive/v2.4.1.zip
!unzip v2.4.1.zip &> 0
!mv tensorflow-2.4.1/ tensorflow/
import sys
# We add this path so we can import the speech processing modules.
sys.path.append("/content/tensorflow/tensorflow/examples/speech_commands/")
import input_data
import models
import numpy as np
import glob
import os
import re
import shutil
from google.colab import files
!pip install ffmpeg-python &> 0
!apt-get update && apt-get -qq install xxd

TensorFlow 1.x selected.
--2022-03-10 00:10:09--  https://github.com/tensorflow/tensorflow/archive/v2.4.1.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/tensorflow/tensorflow/zip/refs/tags/v2.4.1 [following]
--2022-03-10 00:10:09--  https://codeload.github.com/tensorflow/tensorflow/zip/refs/tags/v2.4.1
Resolving codeload.github.com (codeload.github.com)... 140.82.121.10
Connecting to codeload.github.com (codeload.github.com)|140.82.121.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v2.4.1.zip’

v2.4.1.zip              [  <=>               ]  66.13M  9.42MB/s    in 7.2s    

2022-03-10 00:10:17 (9.14 MB/s) - ‘v2.4.1.zip’ saved [69346072]

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.downloa

### Import your collected data
First we are going to download Pete's dataset (Speech Commands dataset) to use as a base set of "other words" and "background noise" that you can build ontop of for your dataset. Doing this will make your model work a lot better, especially if you are training it with a small amount of custom data!

**Note: this may take a couple of minutes to run!**

In [ ]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz
DATASET_DIR =  'dataset/'
!mkdir dataset
!tar -xf speech_commands_v0.02.tar.gz -C 'dataset'
!rm -r -f speech_commands_v0.02.tar.gz

--2022-02-28 16:39:36--  https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.136.128, 142.250.148.128, 209.85.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.136.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2428923189 (2.3G) [application/gzip]
Saving to: ‘speech_commands_v0.02.tar.gz’

speech_commands_v0. 100%[===================>]   2.26G   107MB/s    in 21s     

2022-02-28 16:39:56 (112 MB/s) - ‘speech_commands_v0.02.tar.gz’ saved [2428923189/2428923189]



Now you'll need to upload your all of your custom audio files that you recorded using the Open Speech Recording tool (aka the ```*.ogg``` files). 

**Note: you can select multiple files and upload them all at once!** 

Original dataset includes the following words:

Options for target words are: "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", “backward”, “forward”, “follow”, “learn”,

Additional words that will be used to help train the "unkown" label are: "bed", "bird", "cat", "dog", "happy", "house", "marvin", "sheila", "tree", "wow"

In [ ]:
uploaded = files.upload()

TypeError: ignored

Then we can convert them into correctly trimmed WAV files and then store them in the appropriate folders in the ```DATASET_DIR```.
We will use Pete's extract_loudest_section tool which you can find more documentation about here: https://github.com/petewarden/extract_loudest_section

In [ ]:
# convert the ogg files to wavs
!mkdir wavs
!find *.ogg -print0 | xargs -0 basename -s .ogg | xargs -I {} ffmpeg -i {}.ogg -ar 16000 wavs/{}.wav
!rm -r -f *.ogg

# then use pete's tool to only extract 1 second clips from them for use with the KWS pipeline
!mkdir trimmed_wavs
!git clone https://github.com/petewarden/extract_loudest_section.git
!make -C extract_loudest_section/
!/tmp/extract_loudest_section/gen/bin/extract_loudest_section 'wavs/*.wav' trimmed_wavs/
!rm -r -f /wavs

Streaming output truncated to the last 5000 lines.
    Stream #0:0(eng): Audio: opus, 48000 Hz, mono, fltp (default)
Stream mapping:
  Stream #0:0 -> #0:0 (opus (native) -> pcm_s16le (native))
Press [q] to stop, [?] for help
Output #0, wav, to 'wavs/never_1646443617434 (3).wav':
  Metadata:
    ISFT            : Lavf57.83.100
    Stream #0:0(eng): Audio: pcm_s16le ([1][0][0][0] / 0x0001), 16000 Hz, mono, s16, 256 kb/s (default)
    Metadata:
      encoder         : Lavc57.107.100 pcm_s16le
size=      45kB time=00:00:01.43 bitrate= 256.5kbits/s speed= 251x    
video:0kB audio:45kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.169271%
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --e

In [ ]:
# Store them in the appropriate folders
data_index = {}
os.chdir('trimmed_wavs')
search_path = os.path.join('*.wav')
for wav_path in glob.glob(search_path):
    original_wav_path = wav_path
    parts = wav_path.split('_')
    if len(parts) > 2:
        wav_path = parts[0] + '_' + ''.join(parts[1:])
    matches = re.search('([^/_]+)_([^/_]+)\.wav', wav_path)
    if not matches:
        raise Exception('File name not in a recognized form:"%s"' % wav_path)
    word = matches.group(1).lower()
    instance = matches.group(2).lower()
    if not word in data_index:
      data_index[word] = {}
    if instance in data_index[word]:
        raise Exception('Audio instance already seen:"%s"' % wav_path)
    data_index[word][instance] = original_wav_path

output_dir = os.path.join('..', 'dataset')
try:
    os.mkdir(output_dir)
except:
    pass
for word in data_index:
  word_dir = os.path.join(output_dir, word)
  try:
      os.mkdir(word_dir)
      print('Created dir: ' + word_dir)
  except:
      print('Storing in existing dir: ' + word_dir)
  for instance in data_index[word]:
    wav_path = data_index[word][instance]
    output_path = os.path.join(word_dir, instance + '.wav')
    shutil.copyfile(wav_path, output_path)
os.chdir('..')
!rm -r -f trimmed_wavs

Created dir: ../dataset/all
Created dir: ../dataset/must
Created dir: ../dataset/never
Created dir: ../dataset/only
Created dir: ../dataset/none


You can download your full custom dataset as a zipfile for use later.
Download your files from the Files area by clicking on the three dots next to the zip file. 

Note: this command will take a little while to run as the combination of your data and Speech Commands dataset will be relatively large!

If you would like to create a zip of just your dataset please re-run this colab (or factory reset it to wipe the memory) and skip the line that downloads Pete's dataset.

Finally if you have room in your Google Drive and would like to load and store your dataset from there in the future you can mount your Google Drive in Colab [as described in this blog post](https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166).

In [ ]:
!zip -r myKWSDataset.zip dataset

  adding: dataset/ (stored 0%)
  adding: dataset/never/ (stored 0%)
  adding: dataset/never/1646443569074 (1).wav (deflated 10%)
  adding: dataset/never/1646614522629 (1) (1).wav (deflated 33%)
  adding: dataset/never/1646614559026 (1).wav (deflated 25%)
  adding: dataset/never/1646614559031.wav (deflated 30%)
  adding: dataset/never/1646614522627 (2).wav (deflated 33%)
  adding: dataset/never/1646614607617.wav (deflated 29%)
  adding: dataset/never/1646614522628 (1).wav (deflated 31%)
  adding: dataset/never/1646443617432 (1) (1).wav (deflated 8%)
  adding: dataset/never/1646443569068.wav (deflated 9%)
  adding: dataset/never/1646443617432 (2).wav (deflated 9%)
  adding: dataset/never/1646614522627 (1) (1).wav (deflated 26%)
  adding: dataset/never/1646614559026.wav (deflated 25%)
  adding: dataset/never/1646614607626.wav (deflated 30%)
  adding: dataset/never/1646443617432 (1).wav (deflated 8%)
  adding: dataset/never/1646614607622 (1).wav (deflated 35%)
  adding: dataset/never/16464

## Processing the data

Now let's go to Edge Impulse to train a model that can also be used on the microcontroller
https://www.edgeimpulse.com/

Please create acoount here!